In [1]:
import cv2
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to convert an image to thermal
def convert_to_thermal(image_path):
    image = cv2.imread(image_path)
    thermal_image = cv2.applyColorMap(image, cv2.COLORMAP_JET)
    return Image.fromarray(thermal_image)

# Convert all images in a directory to thermal images
def convert_directory_to_thermal(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                input_path = os.path.join(root, file)
                relative_path = os.path.relpath(input_path, input_dir)
                output_path = os.path.join(output_dir, relative_path)

                # Create output directory if it doesn't exist
                os.makedirs(os.path.dirname(output_path), exist_ok=True)

                thermal_image = convert_to_thermal(input_path)
                thermal_image.save(output_path)

# Paths to original and thermal image directories
train_path = "/Users/niteshyadav/Lyme_Disease/train"
test_path = "/Users/niteshyadav/Lyme_Disease/test"
val_path = "/Users/niteshyadav/Lyme_Disease/val"

thermal_train_path = "/Users/niteshyadav/Lyme_Disease/thermal_img/train"
thermal_test_path = "/Users/niteshyadav/Lyme_Disease/thermal_img/test"
thermal_val_path = "/Users/niteshyadav/Lyme_Disease/thermal_img/val"

# Convert the datasets to thermal images
convert_directory_to_thermal(train_path, thermal_train_path)
convert_directory_to_thermal(test_path, thermal_test_path)
convert_directory_to_thermal(val_path, thermal_val_path)

# Data Augmentation and Data Generator for Thermal Images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    thermal_train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    thermal_val_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Model Architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

# Training the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Evaluate the model on the test set
test_generator = test_datagen.flow_from_directory(
    thermal_test_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test accuracy: {test_acc:.4f}')


Found 4038 images belonging to 2 classes.
Found 506 images belonging to 2 classes.


/Users/niteshyadav/anaconda3/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30


/Users/niteshyadav/anaconda3/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


126/126 ━━━━━━━━━━━━━━━━━━━━ 30s 229ms/step - accuracy: 0.7969 - loss: 0.4904 - val_accuracy: 0.8188 - val_loss: 0.4845
Epoch 2/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step - accuracy: 0.8125 - loss: 0.4942 - val_accuracy: 0.7308 - val_loss: 0.6403
Epoch 3/30


/Users/niteshyadav/anaconda3/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


126/126 ━━━━━━━━━━━━━━━━━━━━ 30s 230ms/step - accuracy: 0.8238 - loss: 0.4668 - val_accuracy: 0.8167 - val_loss: 0.4751
Epoch 4/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 451us/step - accuracy: 0.9375 - loss: 0.2952 - val_accuracy: 0.7692 - val_loss: 0.5667
Epoch 5/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 30s 231ms/step - accuracy: 0.8011 - loss: 0.4995 - val_accuracy: 0.8083 - val_loss: 0.4913
Epoch 6/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 440us/step - accuracy: 0.9062 - loss: 0.3292 - val_accuracy: 0.9231 - val_loss: 0.3132
Epoch 7/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 30s 232ms/step - accuracy: 0.8091 - loss: 0.4863 - val_accuracy: 0.8125 - val_loss: 0.4773
Epoch 8/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 443us/step - accuracy: 0.8750 - loss: 0.3764 - val_accuracy: 0.8462 - val_loss: 0.4518
Epoch 9/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 30s 235ms/step - accuracy: 0.8146 - loss: 0.4780 - val_accuracy: 0.8167 - val_loss: 0.4806
Epoch 10/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - accuracy: 0.7500 - loss: 0.6106 - val_ac

In [5]:
import cv2
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to convert an image to thermal
def convert_to_thermal(image_path):
    image = cv2.imread(image_path)
    thermal_image = cv2.applyColorMap(image, cv2.COLORMAP_JET)
    return Image.fromarray(thermal_image)

# Convert all images in a directory to thermal images
def convert_directory_to_thermal(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                input_path = os.path.join(root, file)
                relative_path = os.path.relpath(input_path, input_dir)
                output_path = os.path.join(output_dir, relative_path)

                # Create output directory if it doesn't exist
                os.makedirs(os.path.dirname(output_path), exist_ok=True)

                thermal_image = convert_to_thermal(input_path)
                thermal_image.save(output_path)

# Paths to original and thermal image directories

#test_path = "/Users/niteshyadav/Lyme_Disease/test"

# Paths to the dataset folders
train_path = "/Users/niteshyadav/Lyme_Disease/Train_rash/Train_2_Cases"
val_path = "/Users/niteshyadav/Lyme_Disease/Validation_rash/Validation_2_Cases"

thermal_train_path = "/Users/niteshyadav/Lyme_Disease/train_thermal_rashes"
#thermal_test_path = "/Users/niteshyadav/Lyme_Disease/thermal_img/test"
thermal_val_path = "/Users/niteshyadav/Lyme_Disease/val_thermal_rashes"

# Convert the datasets to thermal images
convert_directory_to_thermal(train_path, thermal_train_path)
#convert_directory_to_thermal(test_path, thermal_test_path)
convert_directory_to_thermal(val_path, thermal_val_path)

# Data Augmentation and Data Generator for Thermal Images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    thermal_train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    thermal_val_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Model Architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

# Training the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Evaluate the model on the test set
test_generator = test_datagen.flow_from_directory(
    thermal_test_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test accuracy: {test_acc:.4f}')

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50


error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/colormap.cpp:738: error: (-2:Unspecified error) in function 'void cv::colormap::ColorMap::operator()(cv::InputArray, cv::OutputArray) const'
> Not supported (expected: 'src.dims == 2'), where
>     'src.dims' is 0
> must be equal to
>     '2' is 2
